In [53]:
import numpy as np
import pandas as pd

In [54]:
transactions = pd.read_csv('storage/transactions_train.csv')
submission = pd.read_csv('storage/sample_submission.csv')

In [55]:
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

In [56]:
from utils import train_test_split
trn_transactions,val_transactions = train_test_split(transactions)

In [57]:
transactions_3w = trn_transactions[trn_transactions['t_dat'] >= pd.to_datetime('2020-08-24')].copy()
transactions_2w = trn_transactions[trn_transactions['t_dat'] >= pd.to_datetime('2020-08-31')].copy()
transactions_1w = trn_transactions[trn_transactions['t_dat'] >= pd.to_datetime('2020-09-07')].copy()

In [58]:
purchase_dict_3w = {}

for i,x in enumerate(zip(transactions_3w['customer_id'], transactions_3w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_3w:
        purchase_dict_3w[cust_id] = {}
    
    if art_id not in purchase_dict_3w[cust_id]:
        purchase_dict_3w[cust_id][art_id] = 0
    
    purchase_dict_3w[cust_id][art_id] += 1
    
print(len(purchase_dict_3w))

dummy_list_3w = list((transactions_3w['article_id'].value_counts()).index)[:12]

205159


In [59]:
purchase_dict_2w = {}

for i,x in enumerate(zip(transactions_2w['customer_id'], transactions_2w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_2w:
        purchase_dict_2w[cust_id] = {}
    
    if art_id not in purchase_dict_2w[cust_id]:
        purchase_dict_2w[cust_id][art_id] = 0
    
    purchase_dict_2w[cust_id][art_id] += 1
    
print(len(purchase_dict_2w))

dummy_list_2w = list((transactions_2w['article_id'].value_counts()).index)[:12]

149748


In [60]:
purchase_dict_1w = {}

for i,x in enumerate(zip(transactions_1w['customer_id'], transactions_1w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_1w:
        purchase_dict_1w[cust_id] = {}
    
    if art_id not in purchase_dict_1w[cust_id]:
        purchase_dict_1w[cust_id][art_id] = 0
    
    purchase_dict_1w[cust_id][art_id] += 1
    
print(len(purchase_dict_1w))

dummy_list_1w = list((transactions_1w['article_id'].value_counts()).index)[:12]

89435


In [ ]:
pred_df = submission[['customer_id']]
prediction_list = []

dummy_pred = list((transactions_1w['article_id'].value_counts()).index)[:12]

for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    if cust_id in purchase_dict_1w:
        l = sorted((purchase_dict_1w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = l[:12]
        else:
            s = l+dummy_list_1w[:(12-len(l))]
    elif cust_id in purchase_dict_2w:
        l = sorted((purchase_dict_2w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
        s = l[:12]
        else:
            s = l+dummy_list_2w[:(12-len(l))]
    elif cust_id in purchase_dict_3w:
        l = sorted((purchase_dict_3w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = l[:12]
        else:
            s = l+dummy_list_3w[:(12-len(l))]
    else:
        s = dummy_pred
    prediction_list.append(s)

pred_df['prediction'] = prediction_list
print(pred_df.shape)
pred_df.head()

In [10]:
gt_df = val_transactions.groupby('customer_id')['article_id'].agg(lambda x: x.tolist()).reset_index()
gt_df.columns = ['customer_id','ground_truth']

In [11]:
def evaluate_score(pred_df,gt_df,k=12):
    from metric import mapk
    eval_df = gt_df.merge(pred_df,on='customer_id')
    print('map@'+str(k), mapk(eval_df['ground_truth'].tolist(),eval_df['prediction'].tolist()))

In [49]:
evaluate_score(pred_df,gt_df)

map@12 0.02091884628667312
